## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Rikitea,-23.1203,-134.9692,78.82,73,100,18.32,PF,2021-04-01 06:16:11,light rain
1,1,Nouadhibou,20.9310,-17.0347,62.60,88,0,14.97,MR,2021-04-01 06:22:29,clear sky
2,2,Qaanaaq,77.4840,-69.3632,-10.01,67,65,8.19,GL,2021-04-01 06:15:14,broken clouds
3,3,Ushuaia,-54.8000,-68.3000,39.20,75,0,7.52,AR,2021-04-01 06:19:43,clear sky
4,4,Kahului,20.8947,-156.4700,73.40,88,1,12.66,US,2021-04-01 06:19:09,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Rikitea,-23.1203,-134.9692,78.82,73,100,18.32,PF,2021-04-01 06:16:11,light rain
7,7,Busselton,-33.6500,115.3333,80.28,49,100,4.63,AU,2021-04-01 06:19:51,overcast clouds
9,9,Cayenne,4.9333,-52.3333,79.00,94,40,8.46,GF,2021-04-01 06:21:45,scattered clouds
14,14,Bambous Virieux,-20.3428,57.7575,84.99,83,75,9.22,MU,2021-04-01 06:19:52,broken clouds
17,17,Kavaratti,10.5669,72.6420,83.68,68,50,11.18,IN,2021-04-01 06:22:31,scattered clouds
19,19,Alofi,-19.0595,-169.9187,80.60,83,4,9.22,NU,2021-04-01 06:21:05,clear sky
27,27,New Norfolk,-42.7826,147.0587,77.00,32,99,10.00,AU,2021-04-01 06:19:53,overcast clouds
29,29,Atuona,-9.8000,-139.0333,80.74,74,79,13.87,PF,2021-04-01 06:16:26,broken clouds
31,31,Honiara,-9.4333,159.9500,82.71,76,97,2.98,SB,2021-04-01 06:22:32,overcast clouds
32,32,Hobart,-42.8794,147.3294,77.00,25,100,14.97,AU,2021-04-01 06:19:50,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                161
City                   161
Lat                    161
Lng                    161
Max Temp               161
Humidity               161
Cloudiness             161
Wind Speed             161
Country                161
Date                   161
Weather Description    161
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [32]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Weather Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,light rain,78.82,-23.1203,-134.9692,
7,Busselton,AU,overcast clouds,80.28,-33.6500,115.3333,
9,Cayenne,GF,scattered clouds,79.00,4.9333,-52.3333,
14,Bambous Virieux,MU,broken clouds,84.99,-20.3428,57.7575,
17,Kavaratti,IN,scattered clouds,83.68,10.5669,72.6420,
19,Alofi,NU,clear sky,80.60,-19.0595,-169.9187,
27,New Norfolk,AU,overcast clouds,77.00,-42.7826,147.0587,
29,Atuona,PF,broken clouds,80.74,-9.8000,-139.0333,
31,Honiara,SB,overcast clouds,82.71,-9.4333,159.9500,
32,Hobart,AU,overcast clouds,77.00,-42.8794,147.3294,


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
        

In [34]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,light rain,78.82,-23.1203,-134.9692,
7,Busselton,AU,overcast clouds,80.28,-33.6500,115.3333,
9,Cayenne,GF,scattered clouds,79.00,4.9333,-52.3333,
14,Bambous Virieux,MU,broken clouds,84.99,-20.3428,57.7575,
17,Kavaratti,IN,scattered clouds,83.68,10.5669,72.6420,
...,...,...,...,...,...,...,...
648,Poum,NC,overcast clouds,77.72,-20.2333,164.0167,
657,Lishui,CN,overcast clouds,79.68,28.4604,119.9103,
666,Puntian,PH,broken clouds,88.65,7.6089,125.1163,
673,Nara,JP,broken clouds,75.20,34.6851,135.8049,


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [37]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [38]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))